In [1]:
import pandas as pd
from astroquery.simbad import Simbad
from jwst import *

%load_ext autoreload
%autoreload 2
Simbad.add_votable_fields('flux(K)','sp')

max_bex_age = 1000 #This isn't the actually maximum age, but the maximum age above which there isn't much data. 

Directories to change depending on what computer you're on

In [2]:
figure_base_directory = "./Figures_F444W/" #Change this to reflect the project we're working on
bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'

In [3]:
computer = 'max_laptop'
connor = False
maxmb = True

if computer == 'connor':
    base_model_dir="/Users/connorvancil/Desktop/AstroResearch/Data/models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = None
elif computer == 'max_laptop': 
    base_model_dir="/Users/maxwellmb/Data/models/ATMO_2020_models/evolutionary_tracks/ATMO_CEQ/"
    bex_file = '../models/BEX_evol_mags_-2_MH_0.00AC.dat'
elif computer == 'max_desktop':
    base_model_dir = "TBD"
    bex_file = None

Read in the Pearce Table

In [4]:
df = pd.read_csv('../allDiscAnalysisResults.csv')
data_arr = df[['Target','Distance_pc','Age_Myr','PearceMPlt_MJup', 'PearcePltApocentre_arcsec','DiscInnerEdgePeri_au','DiscOuterEdgeApo_au']].to_numpy()
target_dict={}
for row in data_arr:
    target_dict[row[0]]=row[1:]

Query all targets

In [5]:
targets=df['Target'].to_numpy()
queried_params=[];spts=[]
for target in targets:
    result_table = Simbad.query_object(target)
    spts.append(result_table['SP_TYPE'][0])
    queried_params.append([target,result_table['FLUX_K'][0]])

Force Spectral Types to those in ck04models

In [6]:
spts[40]='A5V'
spts[103]='A1V'
for i,spt in enumerate(spts):
    if spt not in lookuptable: 
        spts[i] = match_spt(spt)
[params.append(spts[i]) for i, params in  enumerate(queried_params)]

#adding another dictionary to hold these queried paramters in a more useful format
simbad_vals={}
for star in queried_params:
    simbad_vals[star[0]]=[star[1],star[2]]

Set up some configuration info

In [7]:
configs = {'f250m':  {'instrument':'nircam','instrument_mask':'mask335r'},
            'f300m': {'instrument':'nircam','instrument_mask':'mask335r'}, 
            'f356w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f410m': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f444w': {'instrument':'nircam','instrument_mask':'mask335r'},
            'f1140c':{'instrument':'miri',  'instrument_mask':None},
            'f1550c':{'instrument':'miri',  'instrument_mask':None},
            }
filter_list = list(configs.keys())
filter_list = ['f444w'] #For now only doing this filter because the bex filters only have this filter.

In [21]:
full_detection_list = []

for i, target in enumerate(targets):
# for i, target in enumerate([targets[3]]):
    
    age = target_dict[target][1]
    distance = target_dict[target][0]
    comp_sep = target_dict[target][3] #Companion separation in arcseconds
    comp_mass = target_dict[target][2] #Companions mass in M_Jup

    disk_inner_edge = target_dict[target][4]/distance
    disk_outer_edge = target_dict[target][5]/distance
    
    kmag,spt = simbad_vals[target]
    
    mass_limits_list = []
    sep_list = []
    target_detection_array = []
    almost_detected_array = []

    print("Target {}: {}, {} Myr, {:.0f} pc, kmag={:.1f}".format(i,target,age,distance,kmag))

    if comp_sep < 0.4:
        print("\tCompanion separation for {} is < 0.4 arcseconds, skipping".format(target))
        continue

    for filter_name in filter_list:
        
        instrument = configs[filter_name]['instrument']
        instrument_mask = configs[filter_name]['instrument_mask']

        jwst_mag = get_jwst_mag(spt,kmag,instrument,filter_name,filter_dir="./",jwst_mask=instrument_mask,
                    norm_filter='bessel_k').value

        contrast_curves=read_contrast_curves()
        sep,companion_mags=companion_detection_limit(jwst_mag,filter_name,contrast_curves,plot=False);

        good_seps = sep[np.where(np.isfinite(companion_mags))]

        if comp_sep < np.min(good_seps):
            print("\tCompanion of {:.2f} for {} is closer than the IWA in {}, skipping".format(comp_sep,target,filter_name))
            continue

        model_dir = base_model_dir+model_instrument_directories.get(instrument.lower())
        
        if instrument.lower () == "nircam":
            model_dir+=model_nircam_mask_directories.get(instrument_mask.lower())

        if age < max_bex_age:
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,bex_model_filename=bex_file,
                            plot=False)
        else: 
            mass_limits = generate_mass_curve(age,distance,companion_mags,filter_name,sep,model_dir,
                            plot=False)

        if len(mass_limits) < len(sep):
            sep = sep[:len(mass_limits)]
        detected = detect_companion(sep,mass_limits,comp_sep,comp_mass)
        target_detection_array.append(detected)

        #What if we got 2x deeper?
        almost_detected = detect_companion(sep,mass_limits/2.,comp_sep,comp_mass)
        almost_detected_array.append(almost_detected)

        # print("Was the companion detected? {}".format())
        mass_limits_list.append(mass_limits)
        sep_list.append(sep)

    if np.any(target_detection_array):
        print("\tCOMPANION DETECTED!")
        full_detection_list.append(True)
        figure_directory = figure_base_directory+"/Detected/"
    elif np.any(almost_detected_array): 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Almost_Detected/"
    else: 
        full_detection_list.append(False)
        figure_directory = figure_base_directory+"/Non-Detected/"
    
    #If the companion is too late at all filters then skip
    if len(sep_list) < 1:
        continue

    fig,axis = plt.subplots(1,1,figsize=(12,5))
    for i in range(len(mass_limits_list)):
        plt.semilogy(sep_list[i],mass_limits_list[i],label=filter_list[i])
    plt.title("{}, {} Myr, {} pc, kmag={:.1f}, Detected: {}".format(target,age,distance,kmag,full_detection_list[-1]))
    plt.scatter(comp_sep,comp_mass,label="Predicted Companion")

    ymin, ymax = axis.get_ylim()

    plt.fill_between([disk_inner_edge,disk_outer_edge],[ymin],y2=[ymax],color='k',alpha=0.1)
    plt.xlabel("Separation (arcseconds)")
    plt.ylabel(r"Mass Limit ($M_{Jup}$)")
    plt.legend()
    plt.savefig(figure_directory+target+".png",dpi=200)
    print("Saving to {}".format(figure_directory+target+".png"))
    plt.close()

Target 0: CD-522472, 50.0 Myr, 153 pc, kmag=8.7
	Companion separation for CD-522472 is < 0.4 arcseconds, skipping
Target 1: CPD-722713, 24.0 Myr, 37 pc, kmag=6.9


/Users/maxwellmb/Dropbox (Personal)/Library/Python/contraster/testing/models.py:89: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(masses)[sorted_by_mass], np.array(age_list)[sorted_by_mass], np.array(mag_list)[sorted_by_mass]


	COMPANION DETECTED!
Saving to ./Figures_F444W//Detected/CPD-722713.png
Target 2: GJ581, 5000.0 Myr, 6 pc, kmag=5.8
Saving to ./Figures_F444W//Non-Detected/GJ581.png
Target 3: GJ649, 4500.0 Myr, 10 pc, kmag=5.6
Saving to ./Figures_F444W//Non-Detected/GJ649.png
Target 4: HD166, 250.0 Myr, 14 pc, kmag=4.3


/Users/maxwellmb/Dropbox (Personal)/Library/Python/contraster/testing/models.py:319: RuntimeWarning: Mean of empty slice
  crossover_index = np.argwhere(np.logical_and(np.isfinite(atmo_mass_interp), np.isfinite(bex_mass_interp)))


Saving to ./Figures_F444W//Non-Detected/HD166.png
Target 5: HD203, 24.0 Myr, 40 pc, kmag=5.2
Saving to ./Figures_F444W//Non-Detected/HD203.png
Target 6: HD377, 220.0 Myr, 38 pc, kmag=6.1
Saving to ./Figures_F444W//Non-Detected/HD377.png
Target 7: HD870, 3000.0 Myr, 21 pc, kmag=5.4
Saving to ./Figures_F444W//Non-Detected/HD870.png
Target 8: HD1404, 450.0 Myr, 42 pc, kmag=4.4
Saving to ./Figures_F444W//Non-Detected/HD1404.png
Target 9: HD1461, 6000.0 Myr, 23 pc, kmag=4.9
Saving to ./Figures_F444W//Non-Detected/HD1461.png
Target 10: HD1466, 45.0 Myr, 43 pc, kmag=6.1
	Companion separation for HD1466 is < 0.4 arcseconds, skipping
Target 11: HD2262, 650.0 Myr, 22 pc, kmag=3.5
Saving to ./Figures_F444W//Non-Detected/HD2262.png
Target 12: HD3003, 45.0 Myr, 46 pc, kmag=5.0
	Companion separation for HD3003 is < 0.4 arcseconds, skipping
Target 13: HD3296, 1400.0 Myr, 45 pc, kmag=5.6
Saving to ./Figures_F444W//Non-Detected/HD3296.png
Target 14: HD3670, 42.0 Myr, 77 pc, kmag=7.1
	COMPANION DETECTED

In [11]:
test1 = np.arange(0.,10)
test1[0] = np.nan
test2 = np.arange(20.,30)

combine = np.nanmean(np.vstack([test1,test2]),axis=0)

In [12]:
combine

array([20., 11., 12., 13., 14., 15., 16., 17., 18., 19.])